<a href="https://colab.research.google.com/github/mychele-larson/Big_Data_ETL/blob/main/Big_Data__Books_Part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Book reviews on Amazon - Part 1 Module 22 Assignment - 2nd set of data

In [1]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Hit:7 http://archive.ubuntu.com/ubuntu focal InRelease
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [920 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:12 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,544 kB]
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRele

In [2]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-03-15 00:08:24--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.45MB/s    in 0.2s    

2023-03-15 00:08:24 (5.45 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [5]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles

url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_02.tsv.gz"
spark.sparkContext.addFile(url)
books_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Books_v1_02.tsv.gz"), sep="\t", header=True)

# Drop duplicates
books_df = books_df.drop_duplicates()

# Drop null values
books_df = books_df.dropna()

# Show DataFrame
books_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   52006632|R100JJDVONLM53|0826415725|      15610372|James Brown's Liv...|           Books|          5|            4|          5|   N|                N|it's a history le...|the cuban missile...| 2004-10-04|
|         US|   15014641|R100O96NJ2IORV|038551316X|      86124043|The Book of Mormo...|           Books|          5|    

In [6]:
# Get the number of rows in the DataFrame.
books_df.count()

3105360

##Transform the data

## Create the "review_id_table".

In [7]:
from pyspark.sql.functions import to_date, col
# Create the "review_id_df" DataFrame with the appropriate columns and data types.
# Select the necessary columns

books_review_id_df = books_df.select("review_id", "customer_id", "product_id", "product_parent", to_date("review_date", "yyyy-MM-dd").alias("review_date"))

# Cast "customer_id" and "product_parent" column to integer
books_review_id_df = books_review_id_df.withColumn("customer_id", col("customer_id").cast("integer")).withColumn("product_parent", col("product_parent").cast("integer"))

# Show the resulting DataFrame
books_review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R100JJDVONLM53|   52006632|0826415725|      15610372| 2004-10-04|
|R100O96NJ2IORV|   15014641|038551316X|      86124043| 2004-12-21|
|R101WACVOF1BXP|   53035368|0380973871|     734682067| 2000-10-27|
|R1020BZCVXAWTM|   51239523|0804117950|     109929862| 1999-05-04|
|R102J7OPFNLGF8|   25704917|0689854323|     491653739| 2005-08-17|
|R102O8FX1XT7FC|   40522448|0658011456|     502114843| 2003-05-24|
|R102TBQWPZYV8C|   51281967|1558671951|      38798234| 2002-08-30|
|R1038HO7UOMGB9|   39681662|014028009X|     738261988| 2002-05-15|
|R103LXZ49Q1X0A|   49580275|0375703845|     939433309| 2000-12-20|
|R103UM505R1KFF|   53017408|0375701214|     508561158| 1998-03-23|
|R1041JAMQ22BS2|   37272906|1885394187|     216992833| 2003-09-18|
|R1046IZEKO9W13|   52561712|0374252874|      48810266| 2004-09

## Create the "products" Table

In [8]:
from pyspark.sql.functions import to_date, col, substring
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title" columns. 
books_products_df = books_df.select("product_id", substring(col("product_title"), 1,300).alias("product_title"))

# Drop duplicates
books_products_df = books_products_df.drop_duplicates()

# Show the resulting DataFrame
books_products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0029148510|Acts of War: Beha...|
|1844494896|Waiting For Kate ...|
|0451216954|Dark Lover (Black...|
|006056718X|Often Wrong, Neve...|
|0965852768|101 Spooktacular ...|
|141960435X|Flip Flop Spanish...|
|0471282502|Experimental Orga...|
|0756613647|Universe: The Def...|
|B0006YGCE8|Seismic risk asse...|
|207030602X|L Etranger (Folio...|
|1569711224|Aliens: Hive  (2n...|
|0756613744|Go Figure!: A Tot...|
|0891073825|The Cross: God's ...|
|1595140840|            Teach Me|
|1844491412|Led Zeppelin: The...|
|0060737980|Surrounded by Idi...|
|0486265250|Ancient Egyptian ...|
|0763627798|           Shoe Baby|
|097559950X|Natural Cures "Th...|
|0595362478| Accursed Inhabitant|
+----------+--------------------+
only showing top 20 rows



In [9]:
# Group by columns to find duplicates
duplicates = books_products_df.groupBy("product_id", "product_title").count().filter(col("count") > 1)

# Show duplicates
duplicates.show()

+----------+-------------+-----+
|product_id|product_title|count|
+----------+-------------+-----+
+----------+-------------+-----+



## Create the "customers" Table

In [10]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
books_customers_df = books_df.groupBy("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count").withColumn("customer_id", col("customer_id").cast("integer"))

# Show the resulting DataFrame
books_customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   53045101|            38|
|   37144346|            18|
|   36847515|             3|
|   52894069|             8|
|   41553221|             2|
|   52992798|            61|
|   51930777|            18|
|   51369269|             8|
|   52815943|            26|
|   24180231|             1|
|   49257001|           207|
|   39213659|             1|
|   31056457|             1|
|   51067556|            23|
|   52626661|             1|
|   37695339|             1|
|   44768257|             2|
|   48688230|             4|
|   49737799|             2|
|   52171393|             5|
+-----------+--------------+
only showing top 20 rows



## Create the "vine_table".

In [11]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
books_vine_df = books_df.select("review_id", "star_rating", "helpful_votes", "total_votes", "vine") \
            .withColumn("star_rating", col("star_rating").cast("integer")) \
            .withColumn("helpful_votes", col("helpful_votes").cast("integer")) \
            .withColumn("total_votes", col("total_votes").cast("integer"))
            
# Show the resulting DataFrame
books_vine_df.show()


+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R100JJDVONLM53|          5|            4|          5|   N|
|R100O96NJ2IORV|          5|           19|         30|   N|
|R101WACVOF1BXP|          4|            0|          3|   N|
|R1020BZCVXAWTM|          5|            3|          3|   N|
|R102J7OPFNLGF8|          5|            1|          1|   N|
|R102O8FX1XT7FC|          5|           12|         16|   N|
|R102TBQWPZYV8C|          2|           28|         29|   N|
|R1038HO7UOMGB9|          1|            8|         10|   N|
|R103LXZ49Q1X0A|          5|            0|          2|   N|
|R103UM505R1KFF|          5|            0|          0|   N|
|R1041JAMQ22BS2|          5|            5|          5|   N|
|R1046IZEKO9W13|          5|           12|         19|   N|
|R104CN98MXM9AR|          3|            4|          5|   N|
| R1052KCKIYPKF|          5|            

# Load Postgres Set-up

In [12]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://rds-bigdata.cjysn375vm3e.us-east-2.rds.amazonaws.com:5432/RDS_Database"
config = {"user":"postgres", 
          "password": "postgres", 
          "driver":"org.postgresql.Driver"}

In [13]:
# Write review_id_df to table in RDS
books_review_id_df.write.jdbc(url=jdbc_url, table='books_review_id_table', mode=mode, properties=config)

In [14]:
# Write products_df to table in RDS
books_products_df.write.jdbc(url=jdbc_url, table='books_products', mode=mode, properties=config)


In [15]:
# Write customers_df to table in RDS
books_customers_df.write.jdbc(url=jdbc_url, table='books_customers', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
books_vine_df.write.jdbc(url=jdbc_url, table='books_vine_table', mode=mode, properties=config)